In [1]:
#Implement logistic regression in WEE
using DataFrames, GLM, Distributions, Optim

function WEE_binary(formula::DataFrames.Formula, D, data::DataFrames.DataFrame, pd_pop::Float64, iter::Int64= 10, boot::Int64 = 0)
    mf = ModelFrame(formu, data)
    responseV = formu.lhs # get y (response varible)
    y = mf.df[responseV] # get dataframe[y1]
    
    namesx = DataFrames.allvars(formu)[1:end-1] #get string x1, x2
    
    xx = ModelMatrix(mf)
    
    temp_data = convert(DataFrame, xx.m[:,2:end])
    temp_data[:D] = D
    temp_data[:y] = y
    
    n1 = sum(D.==1)
    n0 = sum(D.==0)
    
    # compute the weight p(D|X)
    formu.lhs = :D
    gamma=coef(glm(formu, temp_data, Binomial()))
    
    function PO(gamma0::Float64)
        gamma[1] = gamma0
        (mean(exp(xx.m * gamma)./(1+exp(xx.m * gamma))) - pd_pop)^2
    end
    
    # Get the argument when PO has the minimum value
    gamma[1] = optimize(PO, -100, 100).minimum
    temp_data[:estpx] = exp(xx.m * gamma)./(1+exp(xx.m * gamma))
    
    #estimate P(Y=1) in cases and controls separately
    formu.lhs = :y
    pyD1 = glm(formu, temp_data[temp_data[:D] .== 1,:], Binomial()) # fit the case
    pyD0 = glm(formu, temp_data[temp_data[:D] .== 0,:], Binomial()) # fit the control
    
    pred1 = predict(pyD1, temp_data[temp_data[:D] .== 1,:]) # generate pseudo control
    pred0 = predict(pyD1, temp_data[temp_data[:D] .== 0,:]) # generate pseudo case
    
    py1 = exp(pred1)./(1+exp(pred1))
    py0 = exp(pred0)./(1+exp(pred0))
    
    # Generate pseudo observations for iter times and get the averaged $iter$ estimates as coefficient pseudo = NULL
    pseudo = DataFrame() # Build an empty dataframe
    for n = 1:size(namesx)[1]
        pseudo[Symbol(namesx[n])] = Float64[]
    end
    
    for iiter = 1:iter
        pseudo1 = [rand(Bernoulli(x)) for x in py1]
        pseudo0 = [rand(Bernoulli(x)) for x in py0]
        data1 = DataFrame(D = repmat([0], n1), y = pseudo1)
        data1[namesx[1]] = temp_data[temp_data[:D] .== 1,:][1]
        data1[namesx[2]] = temp_data[temp_data[:D] .== 1,:][2]
        data1[:estpx] = temp_data[temp_data[:D] .== 1,:][:estpx]
        
        data0 = DataFrame(D = repmat([1], n0), y = pseudo0)
        data0[namesx[1]] = temp_data[temp_data[:D] .== 0,:][1]
        data0[namesx[2]] = temp_data[temp_data[:D] .== 0,:][2]
        data0[:estpx] = temp_data[temp_data[:D] .== 0,:][:estpx]
        
        alldat = vcat(temp_data, data1, data0)
        alldat[alldat[:D] .== 0, :estpx] = 1 - alldat[alldat[:D] .== 0, :estpx]
        
        push!(pseudo, coef(glm(formu, alldat, Binomial(), LogitLink(), wts = convert(Array, alldat[:,:estpx])))[2:end])
    end
    
    # The point estimate
    cf = squeeze(mean(Array(pseudo), 1), 1)
    
    #bootstrap SE
    bootcoef = DataFrame() # Build an empty dataframe
    for n = 1:size(namesx)[1]
        bootcoef[namesx[n]] = Float64[]
    end
    
    if boot == 0
        push!(bootcoef, cf)
        bootcoef
    else
        sample_cases = temp_data[temp_data[:D] .== 1,:]
        sample_controls = temp_data[temp_data[:D] .== 0,:]
        
        for iboot in 1:boot
            boot_cases_sample = sample_cases[sample(1:n1, n1, replace = true), :]
            boot_controls_sample = sample_controls[sample(1:n0, n0, replace = true), :]
            bootsample = vcat(boot_cases_sample, boot_controls_sample)
            
            bootmf = ModelFrame(formu, bootsample)
            bootxx = ModelMatrix(bootmf)
            
            # compute the weight p(D|X) 
            formu.lhs = :D
            gamma = coef(glm(formu, bootsample, Binomial()))
            
            function Boot_PO(gamma0::Float64)
                gamma[1] = gamma0
                (mean(exp(bootxx.m * gamma)./(1+exp(bootxx.m * gamma))) - pd_pop)^2
            end
            
            gamma[1] = optimize(Boot_PO, -100, 100).minimum
            bootsample[:estpx] = exp(bootxx.m * gamma)./(1+exp(bootxx.m * gamma))
            
            formu.lhs = :y
            
            pyD1 = glm(formu, boot_cases_sample, Binomial()) # fit the case
            pyD0 = glm(formu, boot_controls_sample, Binomial()) # fit the control
        
            pred1 = predict(pyD1, boot_cases_sample) # generate pseudo control
            pred0 = predict(pyD1, boot_controls_sample) # generate pseudo case
            
            py1 = exp(pred1)./(1+exp(pred1))
            py0 = exp(pred0)./(1+exp(pred0))
            
            # generate pseudo observations for T(=10) times and get the averaged T estimates as coefficient
            
            pseudo = DataFrame()
            
            for n = 1:size(namesx)[1]
                pseudo[namesx[n]] = Float64[]
            end
            
            for iiter = 1:iter
                pseudo1 = [rand(Bernoulli(x)) for x in py1]
                pseudo0 = [rand(Bernoulli(x)) for x in py0]
                
                data1 = DataFrame(D = repmat([0], n1), y = py1)
                data1[namesx[1]] = boot_cases_sample[1]
                data1[namesx[2]] = boot_cases_sample[2]
                data1[:estpx] = boot_cases_sample[:estpx]
                
                data0 = DataFrame(D = repmat([1], n0), y = py0)
                data0[namesx[1]] = boot_controls_sample[1]
                data0[namesx[2]] = boot_controls_sample[2]
                data0[:estpx] = boot_controls_sample[:estpx]
                
                alldat = vcat(bootsample, data1, data0)
                alldat[alldat[:D] .== 0, :estpx] = 1 - alldat[alldat[:D] .== 0, :estpx]
                
                push!(pseudo, coef(glm(formu, alldat, Normal(), wts = convert(Array, alldat[:,:estpx])))[2:end])
            
            end
            temp_cf = squeeze(mean(Array(pseudo), 1), 1)
            push!(bootcoef, temp_cf)
        end
        var_num = var(Array(bootcoef), 1)
        wald = (cf.^2)./squeeze(var_num, 1)
        pvalue = [ccdf(Chisq(1), m) for m in wald]
        TAB = DataFrame(Variable = namesx, Estimate = cf, StdErr = squeeze(sqrt(var_num), 1), Wald = wald, p_value = pvalue)
    end
end



WEE_binary (generic function with 3 methods)

In [2]:
#Test data
x = DataFrame(x1 = rand(Binomial(2, 0.3), 3000), x2 = rand(Binomial(2, 0.2), 3000))
y1 = rand(Binomial(1, 0.3), 3000)
D = vcat([0 for i = 1:1000], [1 for i = 1:2000])   #vcat(repmat([0], 1000), repmat([1], 2000)) /// repeat([1, 2, 3, 4], outer=[2])
y1d1 = DataFrame(y1 = y1, D = D)

data = hcat(x, y1d1)
pd = 0.1
formu = y1 ~ x1 + x2
boot = 10
pd_pop = 0.1
iter = 10

WEE_binary(formu, D, data, pd_pop, iter, boot)

,Variable,Estimate,StdErr,Wald,p_value
1,x1,-0.0479605381130529,0.007974114017501603,36.17455623393674,1.8041073429611087e-9
2,x2,0.006790812804044966,0.010540310221231164,0.4150847054892678,0.5193999267323524


In [74]:
#Input formu, D, data, pd_pop, iter

mf = ModelFrame(formu, data)
responseV = formu.lhs # get y (response varible)
y = mf.df[responseV] # get dataframe[y1]

namesx = DataFrames.allvars(formu)[1:end-1] #get string x1, x2

xx = ModelMatrix(mf)

temp_data = convert(DataFrame, xx.m[:,2:end])
temp_data[:D] = D
temp_data[:y] = y

n1 = sum(D.==1)
n0 = sum(D.==0)

# compute the weight p(D|X)
formu.lhs = :D
gamma=coef(glm(formu, temp_data, Binomial()));


In [5]:
function PO(gamma0::Float64)
    gamma[1] = gamma0
    (mean(exp(xx.m * gamma)./(1+exp(xx.m * gamma))) - pd_pop)^2
end

PO (generic function with 1 method)

In [75]:
# Get the argument when PO has the minimum value
gamma[1] = optimize(PO, -100, 100).minimum
temp_data[:estpx] = exp(xx.m * gamma)./(1+exp(xx.m * gamma));

In [76]:
#estimate P(Y=1) in cases and controls separately
formu.lhs = :y
pyD1 = glm(formu, temp_data[temp_data[:D] .== 1,:], Binomial()) # fit the case
pyD0 = glm(formu, temp_data[temp_data[:D] .== 0,:], Binomial()) # fit the control

pred1 = predict(pyD1, temp_data[temp_data[:D] .== 1,:]) # generate pseudo control
pred0 = predict(pyD1, temp_data[temp_data[:D] .== 0,:]) # generate pseudo case

py1 = exp(pred1)./(1+exp(pred1))
py0 = exp(pred0)./(1+exp(pred0));

In [91]:
# Generate pseudo observations for iter times and get the averaged $iter$ estimates as coefficient pseudo = NULL
pseudo = DataFrame() # Build an empty dataframe
for n = 1:size(namesx)[1]
    pseudo[Symbol(namesx[n])] = Float64[]
end

for iiter = 1:iter
    pseudo1 = [rand(Bernoulli(x)) for x in py1]
    pseudo0 = [rand(Bernoulli(x)) for x in py0]
    data1 = DataFrame(D = repmat([0], n1), y = pseudo1)
    data1[namesx[1]] = temp_data[temp_data[:D] .== 1,:][1]
    data1[namesx[2]] = temp_data[temp_data[:D] .== 1,:][2]
    data1[:estpx] = temp_data[temp_data[:D] .== 1,:][:estpx]
    
    data0 = DataFrame(D = repmat([1], n0), y = pseudo0)
    data0[namesx[1]] = temp_data[temp_data[:D] .== 0,:][1]
    data0[namesx[2]] = temp_data[temp_data[:D] .== 0,:][2]
    data0[:estpx] = temp_data[temp_data[:D] .== 0,:][:estpx]
    
    alldat = vcat(temp_data, data1, data0)
    alldat[alldat[:D] .== 0, :estpx] = 1 - alldat[alldat[:D] .== 0, :estpx]
    
    push!(pseudo, coef(glm(formu, alldat, Binomial(), LogitLink(), wts = convert(Array, alldat[:,:estpx])))[2:end])
end

pseudo


,x1,x2
1,-0.0029755816384571523,-0.04168238872694586
2,0.03860479404966801,0.05088982974265588
3,-0.0036291585805375533,-0.0007759702386229952
4,0.04744825562341582,0.07577215035708201
5,0.0044390371698287395,-0.035621858147241316
6,-0.008020107251139162,0.04379956254568457
7,0.004002352498146907,-0.02888064271966861
8,0.016569324197047586,-0.03672601207560112
9,-0.06050854924053173,0.009409009185177944
10,0.015073455977836738,0.032640854500089606


In [122]:
# The point estimate
cf = squeeze(mean(Array(pseudo), 1), 1)

#bootstrap SE
bootcoef = DataFrame() # Build an empty dataframe
for n = 1:size(namesx)[1]
    bootcoef[namesx[n]] = Float64[]
end

if boot == 0
    push!(bootcoef, cf)
    bootcoef
else
    sample_cases = temp_data[temp_data[:D] .== 1,:]
    sample_controls = temp_data[temp_data[:D] .== 0,:]
        
    for iboot in 1:boot
        
        boot_cases_sample = sample_cases[sample(1:n1, n1, replace = true), :]
        boot_controls_sample = sample_controls[sample(1:n0, n0, replace = true), :]
        bootsample = vcat(boot_cases_sample, boot_controls_sample)
        
        bootmf = ModelFrame(formu, bootsample)
        bootxx = ModelMatrix(bootmf)
        
        # compute the weight p(D|X) 
        formu.lhs = :D
        gamma = coef(glm(formu, bootsample, Binomial()))
        
        function Boot_PO(gamma0::Float64)
            gamma[1] = gamma0
            (mean(exp(bootxx.m * gamma)./(1+exp(bootxx.m * gamma))) - pd_pop)^2
        end
        
        gamma[1] = optimize(Boot_PO, -100, 100).minimum
        bootsample[:estpx] = exp(bootxx.m * gamma)./(1+exp(bootxx.m * gamma))
        
        formu.lhs = :y
        
        pyD1 = glm(formu, boot_cases_sample, Binomial()) # fit the case
        pyD0 = glm(formu, boot_controls_sample, Binomial()) # fit the control
        
        pred1 = predict(pyD1, boot_cases_sample) # generate pseudo control
        pred0 = predict(pyD1, boot_controls_sample) # generate pseudo case
        
        py1 = exp(pred1)./(1+exp(pred1))
        py0 = exp(pred0)./(1+exp(pred0))
        
        # generate pseudo observations for T(=10) times and get the averaged T estimates as coefficient
        
        pseudo = DataFrame()
        
        for n = 1:size(namesx)[1]
            pseudo[namesx[n]] = Float64[]
        end
        
        for iiter = 1:iter
            pseudo1 = [rand(Bernoulli(x)) for x in py1]
            pseudo0 = [rand(Bernoulli(x)) for x in py0]
            
            data1 = DataFrame(D = repmat([0], n1), y = py1)
            data1[namesx[1]] = boot_cases_sample[1]
            data1[namesx[2]] = boot_cases_sample[2]
            data1[:estpx] = boot_cases_sample[:estpx]
            
            data0 = DataFrame(D = repmat([1], n0), y = py0)
            data0[namesx[1]] = boot_controls_sample[1]
            data0[namesx[2]] = boot_controls_sample[2]
            data0[:estpx] = boot_controls_sample[:estpx]
            
            alldat = vcat(bootsample, data1, data0)
            alldat[alldat[:D] .== 0, :estpx] = 1 - alldat[alldat[:D] .== 0, :estpx]
            
            push!(pseudo, coef(glm(formu, alldat, Normal(), wts = convert(Array, alldat[:,:estpx])))[2:end])
        
        end
        temp_cf = squeeze(mean(Array(pseudo), 1), 1)
        push!(bootcoef, temp_cf)
    end
    var_num = var(Array(bootcoef), 1)
    wald = (cf.^2)./squeeze(var_num, 1)
    pvalue = [ccdf(Chisq(1), m) for m in wald]
    TAB = DataFrame(Variable = namesx, Estimate = cf, StdErr = squeeze(sqrt(var_num), 1), Wald = wald, p_value = pvalue)
end

,Variable,Estimate,StdErr,Wald,p_value
1,x1,0.010210842024657137,0.006021422571450194,2.8755763416238063,0.08993277261934657
2,x2,0.012030490774245969,0.00820226019045936,2.1512932038738155,0.14244986074878022
